<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/master/2%20Advanced/065Voronoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 065 Voronoi

[Voronoiの紹介スライド](https://github.com/vitroid/PythonTutorials/blob/master/2%20Advanced/Voronoi.pdf?raw=true)


In [ ]:
# 県庁所在地の座標
# https://www.benricho.org/chimei/latlng_data.html

import numpy as np
pos = np.fromstring("""
43.06417	141.34694
40.82444	140.74
39.70361	141.1525
38.26889	140.87194
39.71861	140.1025
38.24056	140.36333
37.75	140.46778
36.34139	140.44667
36.56583	139.88361
36.39111	139.06083
35.85694	139.64889
35.60472	140.12333
35.68944	139.69167
35.44778	139.6425
37.90222	139.02361
36.69528	137.21139
36.59444	136.62556
36.06528	136.22194
35.66389	138.56833
36.65139	138.18111
35.39111	136.72222
34.97694	138.38306
35.18028	136.90667
34.73028	136.50861
35.00444	135.86833
35.02139	135.75556
34.68639	135.52
34.69139	135.18306
34.68528	135.83278
34.22611	135.1675
35.50361	134.23833
35.47222	133.05056
34.66167	133.935
34.39639	132.45944
34.18583	131.47139
34.06583	134.55944
34.34028	134.04333
33.84167	132.76611
33.55972	133.53111
33.60639	130.41806
33.24944	130.29889
32.74472	129.87361
32.78972	130.74167
33.23806	131.6125
31.91111	131.42389
31.56028	130.55806
26.2125	127.68111
""", sep=" ").reshape(-1,2)[:,::-1]
# 空白区切りのテキストを読みこみ、2列の二次元配列に変換し、2つの行をいれかえる

pos

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure()
ax = plt.axes()

plt.scatter(pos[:,0], pos[:,1])
plt.axis("square")
plt.show()

県庁所在地をつないだDelaunay図

In [ ]:
# Delaunay triangulation
from scipy.spatial import Delaunay
tri = Delaunay(pos)
plt.triplot(pos[:,0], pos[:,1], tri.simplices)
plt.axis("square")
plt.show()

2次元のボロノイ図を描くのはライブラリにまかせる。

In [ ]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(pos)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.triplot(pos[:,0], pos[:,1], tri.simplices, c='red')
#plt.xlim(127.5,145)
plt.ylim(25, 45)
plt.axis([127.5, 145, 25, 45])
plt.gca().set_aspect('equal', adjustable='box')
fig.set_size_inches(11,11)
plt.show()

歪んで見えるのは、縦と横の目盛が違うから。

なお、球面(地球)上の点の場合には、非ユークリッドVoronoi分割を適用する必要がある。(二等分線が大円になる)

もうちょっと少ない点で。例えば4点の場合。

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d
points = np.array([[0, -0.1], [0, 1.1], [1.2, 0], [1, 1]])
vor = Voronoi(points)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.gca().set_aspect('equal', adjustable='box')
fig.set_size_inches(8,8)
plt.show()

vorにはどんな情報が含まれているか。

In [ ]:
vor.__dict__

_pointsが与えた点(4つ)、verticeがvoronoi頂点(外接円の中心)のようだ。よく見ると、辺の情報も含まれている。

 3次元でも4次元でもscipy.spatial.Voronoiを使えばVoronoi分割できるが、残念なことに、既存の関数を使って可視化できるのは2次元まで。
 
 別の例。ひまわりの種の配列。

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

tau  = (5**0.5-1)/2 # Golden ratio
A    = 2*np.pi / tau   # Golden angle

# 400点の角度と半径を一気に計算する。
# 半径を等間隔にすると、あとになるほど種の面積が大きくなるので、
# 半径は番号の平方根に比例するようにとる。
angles = np.array([A*i for i in range(400)])
radii  = np.array([i**0.5 for i in range(400)])

x = radii * np.cos(angles)
y = radii * np.sin(angles)

plt.scatter(x,y)
plt.axis('square')
plt.show()

In [ ]:
pos = np.vstack([x,y]).T

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.voronoi_plot_2d.html
from scipy.spatial import Voronoi, voronoi_plot_2d
vor = Voronoi(pos)
fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',
                line_width=2, line_alpha=0.6, point_size=2)
plt.axis('equal')
plt.xlim(-20,20)
plt.ylim(-20,20)
plt.show()

# Delaunay分割を利用した補間


In [ ]:
from scipy.spatial import Delaunay
import numpy as np

# 郵便局の場所。4つだけ。
points = np.array([[0, -0.1], [0, 1.1], [1.2, 0], [1, 1]])

# いきなりDelaunay三角形分割
tri = Delaunay(points)
tri


`tri`の中身を見る。


In [ ]:
tri.__dict__


三角形を描く。


In [ ]:
import matplotlib.pyplot as plt
plt.triplot(points[:,0], points[:,1], tri.simplices)
plt.scatter(points[:,0], points[:,1], s=100)
plt.axis('square')
plt.show()


外接円の半径と中心は?


In [ ]:
import numpy as np
def plane(p1,p2,p3):
    "p1,p2,p3を通る平面z=ax+by+cの係数a,b,cを返す"
    x1,y1,z1 = p1
    x2,y2,z2 = p2
    x3,y3,z3 = p3
    A = np.array([[x1,y1,1],[x2,y2,1],[x3,y3,1]])
    X = np.array([z1,z2,z3])
    # Aの逆行列とXのドット積
    abc = np.linalg.inv(A) @ X
    return abc

def center_and_radius(a,b,c):
    "平面z=ax+by+cと放物面z=x**2+y**2の交線のxy平面への射影"
    radius = (a**2 + b**2 + 4*c)**0.5/2
    center = (a/2, b/2)
    return center, radius

# tripletは最初のDelaunay三角形の3つの頂点の番号
triplet = tri.simplices[0]
# p3は3つの頂点の座標(x,y)
p3 = points[triplet]
# 放物面にもちあげたz座標をくっつけ、plane()関数にわたす。
a,b,c = plane([p3[0,0], p3[0,1], p3[0,0]**2+p3[0,1]**2],
            [p3[1,0], p3[1,1], p3[1,0]**2+p3[1,1]**2],
            [p3[2,0], p3[2,1], p3[2,0]**2+p3[2,1]**2])
center, radius = center_and_radius(a,b,c)

import matplotlib.pyplot as plt
plt.triplot(points[:,0], points[:,1], tri.simplices)
plt.scatter(points[:,0], points[:,1], s=100)

circle1 = plt.Circle(center, radius, color='r', fill=False)
plt.scatter(center[0], center[1])
plt.gca().add_patch(circle1)

plt.axis('square')
plt.show()



外接円を全部まとめて描く。


In [ ]:
import matplotlib.pyplot as plt
plt.triplot(points[:,0], points[:,1], tri.simplices)
plt.scatter(points[:,0], points[:,1], s=100)

# tripletは最初のDelaunay三角形の3つの頂点の番号
for triplet in tri.simplices:
    # p3は3つの頂点の座標(x,y)
    p3 = points[triplet]
    # 放物面にもちあげたz座標をくっつけ、plane()関数にわたす。
    a,b,c = plane([p3[0,0], p3[0,1], p3[0,0]**2+p3[0,1]**2],
                [p3[1,0], p3[1,1], p3[1,0]**2+p3[1,1]**2],
                [p3[2,0], p3[2,1], p3[2,0]**2+p3[2,1]**2])
    center, radius = center_and_radius(a,b,c)

    # add a circumcircle
    circle1 = plt.Circle(center, radius, color='r', fill=False)
    plt.gca().add_patch(circle1)
    # center dot
    plt.scatter(center[0], center[1])

plt.axis('square')
plt.show()



てきとうに選んだ3つの家`(0.1, 0.2), (1.5, 0.5), (0.5, 1.05)`が、どちらの郵便局のテリトリー内(三角形の中)にあるかを判定する。


In [ ]:
vertices = np.array([(0.1, 0.2), (1.5, 0.5), (0.5, 1.05)])
tri.find_simplex(vertices)


最初と最後の点は三角形1の中にあり、2番目の点はどの三角にも属していない。

三角形1の中で、3つの点がどんな位置にあるか(混合比)を求める。(詳細は理解していません。transformって何?)


In [ ]:
# 以下のリンク先を参考にしました。
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Delaunay.html

# b = tri.transform[1,:2].dot(np.transpose(p - tri.transform[1,2]))
b = tri.transform[1,:2] @  (points - tri.transform[1,2]).T
np.c_[b.T, 1 - b.sum(axis=0)]


1行目は、混合比が全部0以上1以下、つまり三角形1の内部にある。
2行目は負の数や1を越える数があり、三角形1の外の点である。
3行目は成分が0の項があるので、点は三角形の辺の上にある。

理解していないtransformに頼るのではなく、自力で混合比を求めてみる。

三角形OABの中の点Cの内挿は、$OA=a, OB=b, OC=c,$と書き、$pa+qb=c$となる係数$p,q$を求めることにほかならない。成分で書けば、
$$(p,q)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)=(c_x, c_y)$$
書きかえれば、
$$(p,q)=(c_x, c_y)\left(\begin{matrix}a_x, a_y\\ b_x,b_y\end{matrix}\right)^{-1}$$
これをPython化するとこんな感じになる。


In [ ]:
def interpolate(point, triangle):
    # last point is the origin
    o = triangle[2]
    ab = triangle[:2] - o
    print(ab)
    c = point - o
    p, q = np.linalg.inv(ab) @ c
    r = 1 - p - q
    return p, q, r


for point in points:
    p, q, r = interpolate(point, tri.simplices[0])
    print(p, q, r)




    



4つの頂点に、色をあてはめ、それ以外の点はその4色の混合で表す。


In [ ]:
colors = np.array([[1,0,0],
                   [0.5,1,0],
                   [0,1,1],
                   [0.5,0,1]])

import matplotlib.pyplot as plt
plt.scatter(points[:,0], points[:,1], c=colors, s=100)

plt.axis('square')
plt.show()


In [ ]:
# 0.1間隔で
p = []
for ix in range(13):
    for iy in range(11):
        x, y = ix/10, iy/10
        p = np.array([[x, y]])
        # 点がどっちの三角形に属するか。
        which = tri.find_simplex(p)[0]
        # どの三角にも属しない場合は飛ばす。
        if which < 0:
            continue
        # 混合比
        b = tri.transform[which,:2] @ (p - tri.transform[which,2]).T
        mix = np.c_[b.T, 1 - b.sum(axis=0)][0]
        # その三角形を構成する頂点の番号を割りだす。
        v = tri.simplices[which]
        # 
        # その比率で、頂点の色を混ぜる。
        # mixedcolor = mix[0]*colors[v[0]] + mix[1]*colors[v[1]] + mix[2]*colors[v[2]]
        # simpler expression
        mixedcolor = mix @ colors[v]
        mixedcolor = np.around(mixedcolor, decimals=4)
        plt.scatter(x, y, s=50, facecolor=mixedcolor)
plt.triplot(points[:,0], points[:,1], tri.simplices)
plt.scatter(points[:,0], points[:,1], s=100, facecolor=colors)
plt.show()


## これは何に使えるの?

スライドでも紹介した通り、Voronoi分割のアイディアは、情報を幾何学的に扱ういろんなところに顔を出します。また、2次元の情報の処理のために、一旦三次元に持ちあげて考える、というのも非常に面白い視点だと思います。

* データの補間
* データの分類と最適化
* データの可視化

など。アイディアは4次元以上のデータでも使えるはずで、機械学習の手法としても面白いと思います。
